In [1]:
from collections import OrderedDict
import time
import os
import logging

from qiskit.optimization.algorithms import CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from dwave.system import LeapHybridSampler

from helpers import create_models, create_dwave_meo

In [2]:
TEST_DIR = 'TEST_DATA' + "/" + time.strftime("%d_%m_%Y") + "/SPARSE/"
RES_DIR = 'RESULTS' + "/" + time.strftime("%d_%m_%Y") + "/SPARSE/"
os.makedirs(RES_DIR, exist_ok=True)
TEST_DIR

'TEST_DATA/16_03_2021/SPARSE/'

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
qps = create_models(TEST_DIR)

In [5]:
# init Optimizers
cplex = CplexOptimizer()
hybrid = create_dwave_meo(LeapHybridSampler())
dwave = create_dwave_meo(num_reads=1000)

2021-03-16 16:19:28,433 dwave.cloud.client INFO MainThread Fetching definitions of all available solvers
2021-03-16 16:19:32,734 dwave.cloud.client INFO MainThread Adding solver BQMSolver(id='hybrid_binary_quadratic_model_version2')
2021-03-16 16:19:32,737 dwave.cloud.client INFO MainThread Adding solver DQMSolver(id='hybrid_discrete_quadratic_model_version1')


In [6]:
results = OrderedDict()
count_fail_hybrid, count_fail_dwave = 0, 0

for qp_name, qp in qps.items() :
    
    print(qp_name)
    print("number of qubits: ", qp.qubo.get_num_vars())
    output_file_handler = logging.FileHandler(filename=RES_DIR + qp.name + ".log")
    logger.addHandler(output_file_handler)
    
    with open(RES_DIR + qp.name + '.res', 'w') as file:
        file.write(str("Start " + qp.name + "\n " + str(qp.qubo.get_num_vars()) + " qubits needed"))  
        file.write("\n Leap: \n")
        logger.info("\n Leap: \n ")
        res_classic = cplex.solve(qp)
        
        res_hybrid = hybrid.solve(qp)
        problem_id = str(res_hybrid.min_eigen_solver_result.sampleset.info['problem_id'])
        logger.info("\n problem id " + problem_id)
        file.write("\n problem id: " + problem_id)
        if res_hybrid.status != OptimizationResultStatus.SUCCESS:
            file.write("\n No solution found with DWave Hybrid Sampler Leap.\n")
            file.write("\n CPLEX:\n")
            file.write(str(res_classic))
            count_fail_hybrid = count_fail_hybrid + 1
            if count_fail_hybrid > 2 :
                break
        else:
            print("Leap successful!")
            count_fail_hybrid = 0
            
            if res_hybrid.fval == res_classic.fval:
                file.write("\n Leap found optimal solution\n")

            else:
                print("\n optimal value Leap   "+str(res_hybrid.fval) + "  ,  cplex:"+ str(res_classic.fval))
                file.write("\n Leap: \n")
                file.write(str(res_hybrid))
                file.write("\n CPLEX:\n")
                file.write(str(res_classic))
                
            
            
            if count_fail_dwave <= 2:
                file.write("\n DWave Quantum: \n")
                logger.info("\n DWave Quantum: \n ")
                try:
                    res_quantum = dwave.solve(qp)
                    problem_id = str(res_quantum.min_eigen_solver_result.sampleset.info['problem_id'])
                    logger.info("\n problem id " + problem_id)
                    file.write("\n problem id: " + problem_id)

                    if res_quantum.status != OptimizationResultStatus.SUCCESS:
                        print("Dwave quantum solver found no solution.")
                        file.write("\n No solution found with DWave Quantum Solver \n")
                        count_fail_dwave = count_fail_dwave + 1
                        if count_fail_dwave == 3:
                            file.write("\n Stop testing DWave Quantum Solver \n")
                    else:
                        print("Dwave Quantum successful!")
                        count_fail_dwave = 0
                        
                        if res_quantum.fval == res_classic.fval:
                            file.write("\n DWave quantum solver found optimal solution\n")
                        else:
                            print("\n optimal value Dwave quantum "+str(res_quantum.fval) + "  ,  cplex:"+ str(res_classic.fval))
                            file.write("\n DWave quantum solver:\n")
                            file.write(str(res_quantum))
                            file.write("\n CPLEX:\n")
                            file.write(str(res_classic))

                except Exception as ex:
                    print(qp_name, " ", type(ex).__name__, " : ", ex)
                    file.write("\n DWave Solver produced an exception:\n")
                    file.write(str(ex))
                
    logger.removeHandler(output_file_handler)

# see Class SubstituteVariables in https://qiskit.org/documentation/_modules/qiskit/optimization/problems/quadratic_program.html#QuadraticProgram
# for logger warning constraint A0_leb0 is infeasible due to substitution
# Afterwards a check for feasability of substituted solution is done to set OptimizationResultStatus

test_10
number of qubits:  70
Leap successful!

 optimal value Leap   -3.0  ,  cplex:-6.0


2021-03-16 16:20:00,538 dwave.cloud.client INFO MainThread Fetching definitions of all available solvers
2021-03-16 16:20:04,571 dwave.cloud.client INFO MainThread Adding solver StructuredSolver(id='DW_2000Q_6')
2021-03-16 16:20:04,652 dwave.cloud.client INFO MainThread Adding solver StructuredSolver(id='Advantage_system1.1')


Dwave Quantum successful!

 optimal value Dwave quantum 4.0  ,  cplex:-6.0
test_13
number of qubits:  100
Leap successful!

 optimal value Leap   -14.0  ,  cplex:-18.0
Dwave Quantum successful!

 optimal value Dwave quantum -2.0  ,  cplex:-18.0
test_16
number of qubits:  125
Leap successful!

 optimal value Leap   -2.0  ,  cplex:-19.0
Dwave quantum solver found no solution.
test_19
number of qubits:  141
Leap successful!

 optimal value Leap   0.0  ,  cplex:-20.0
Dwave quantum solver found no solution.
test_22
number of qubits:  166
Leap successful!

 optimal value Leap   -11.0  ,  cplex:-21.0
Dwave quantum solver found no solution.
test_25
number of qubits:  172
test_28
number of qubits:  217
Leap successful!

 optimal value Leap   0.0  ,  cplex:-31.0
test_31
number of qubits:  233
test_34
number of qubits:  255
test_37
number of qubits:  285
